Testing coordinate trafos with sparse meshes
=================

In [1]:
from __future__ import print_function
import time
import numpy as np
from sys import getsizeof


In [28]:
import sys
print(sys.long_info)

times = {}
sizes = {}

Nr,Nc,Np = (11,12,13) # number of points per dim
rarr = np.linspace(0.0,10.0,Nr)
carr = np.linspace(-1.0,1.0,Nc)
parr = np.linspace(0.0,2.0*np.pi,Np)


sys.long_info(bits_per_digit=30, sizeof_digit=4)


Build default mesh - spherical coords
-----------

In [29]:
start = time.time()
Rd,Cd,Pd = np.meshgrid(rarr,carr,parr,indexing='ij')
times['dMesh'] = time.time() - start
sizes['dMeshR'] = getsizeof(Rd)
sizes['dMeshC'] = getsizeof(Cd)
sizes['dMeshP'] = getsizeof(Pd)
print("time build default meshgrid:",times['dMesh'])
print("size build default meshgrid: %1.4e %1.4e %1.4e"%(sizes['dMeshR'],sizes['dMeshC'],sizes['dMeshP']))
print("R shape:",Rd.shape)
print("C shape:",Cd.shape)
print("P shape:",Pd.shape)

time build default meshgrid: 0.00523996353149
size build default meshgrid: 1.3856e+04 1.3856e+04 1.3856e+04
R shape: (11, 12, 13)
C shape: (11, 12, 13)
P shape: (11, 12, 13)


Build sparse mesh - spherical coords
-----------

In [30]:
start = time.time()
Rs,Cs,Ps = np.meshgrid(rarr,carr,parr,sparse = True,indexing='ij')
times['sMesh'] = time.time() - start
sizes['sMeshR'] = getsizeof(Rs)
sizes['sMeshC'] = getsizeof(Cs)
sizes['sMeshP'] = getsizeof(Ps)
print("time build default meshgrid:",times['sMesh'])
print("size build default meshgrid: %1.4e %1.4e %1.4e"%(sizes['sMeshR'],sizes['sMeshC'],sizes['sMeshP']))
print("R shape:",Rs.shape)
print("C shape:",Cs.shape)
print("P shape:",Ps.shape)

time build default meshgrid: 0.000257015228271
size build default meshgrid: 2.1600e+02 2.2400e+02 2.3200e+02
R shape: (11, 1, 1)
C shape: (1, 12, 1)
P shape: (1, 1, 13)


Build cartesian coords from spherical
----------

In [96]:
def trafo(R,C,P):
    """
    transformation from spherical to cartesian coordinates
    """
    S=np.sin(np.arccos(C))
    return (R*S*np.cos(P),R*S*np.sin(P),R*C)

Build default mesh - cartesian coords
-----------

In [97]:
start = time.time()
Xd,Yd,Zd = trafo(Rd,Cd,Pd)
times['dTrafo'] = time.time() - start
print("time trafo default meshgrid:",times['dTrafo'])
sizes['dMeshX'] = getsizeof(Xd)
sizes['dMeshY'] = getsizeof(Yd)
sizes['dMeshZ'] = getsizeof(Zd)

print("size trafo default meshgrid: %1.4e %1.4e %1.4e"%(sizes['dMeshX'],sizes['dMeshY'],sizes['dMeshZ']))
print("X shape:",Xd.shape)
print("Y shape:",Yd.shape)
print("Z shape:",Zd.shape)

time trafo default meshgrid: 0.00113201141357
size trafo default meshgrid: 1.3856e+04 1.3856e+04 1.3856e+04
X shape: (11, 12, 13)
Y shape: (11, 12, 13)
Z shape: (11, 12, 13)


Build sparse mesh - cartesian coords
-----------

In [98]:
start = time.time()
Xs,Ys,Zs = trafo(Rs,Cs,Ps)
times['sTrafo'] = time.time() - start
print("time trafo default meshgrid:",times['sTrafo'])
sizes['sMeshX'] = getsizeof(Xs)
sizes['sMeshY'] = getsizeof(Ys)
sizes['sMeshZ'] = getsizeof(Zs)

print("size trafo default meshgrid: %1.4e %1.4e %1.4e"%(sizes['sMeshX'],sizes['sMeshY'],sizes['sMeshZ']))
print("X shape:",Xs.shape)
print("Y shape:",Ys.shape)
print("Z shape:",Zs.shape)

time trafo default meshgrid: 0.00398516654968
size trafo default meshgrid: 1.3856e+04 1.3856e+04 1.1840e+03
X shape: (11, 12, 13)
Y shape: (11, 12, 13)
Z shape: (11, 12, 1)


Conditions on meshgrids - cartesian
-----------

In [108]:
def cond(x,y,z):
    return (x>0)&(y>0)&(z>0)

def condSingle(x):
    return x>0

default mesh
-------

In [109]:
start = time.time()
dCond = cond(Xd,Yd,Zd)
times['dCond'] = time.time() - start
sizes['dCond'] = getsizeof(dCond)
print("time cond default mesh: ",times['dCond'])
print("size cond default mesh: ",sizes['dCond'])
print("dCond shape:",dCond.shape)


time cond default mesh:  0.000225067138672
size cond default mesh:  1844
dCond shape: (11, 12, 13)


sparse mesh
-------

In [111]:
start = time.time()
#sCond = cond(Xs,Ys,Zs)
sCondX = condSingle(Xs)
sCondY = condSingle(Ys)
sCondZ = condSingle(Zs)
sCond = sCondX & sCondY & sCondZ
times['sCond'] = time.time() - start
sizes['sCond'] = getsizeof(sCond)
print("time cond sparse mesh: ",times['sCond'])
print("size cond sparse mesh: ",sizes['sCond'])
print("sCond shape:",sCond.shape)
print("sCondX shape:",sCondX.shape)
print("sCondY shape:",sCondY.shape)
print("sCondZ shape:",sCondZ.shape)
#for i,el in enumerate(sCond):
#    print("el:",i)
#    for i2,el2 in enumerate(el):
#        print("\tel2:",i2)
#        print("\t\t",el2)

time cond sparse mesh:  0.000406980514526
size cond sparse mesh:  1844
sCond shape: (11, 12, 13)
sCondX shape: (11, 12, 13)
sCondY shape: (11, 12, 13)
sCondZ shape: (11, 12, 1)


compare condtions
--------

In [102]:
print("Is there any False in dCond: %s"%(not dCond.all()))
print("Is there any True in dCond: %s"%(sCond.any()))
print("Is there any False in sCond: %s"%(not dCond.all()))
print("Is there any True in sCond: %s"%(sCond.any()))

print("Testing equality:")
if np.array_equiv(dCond,sCond):
    print("\tThe conditions are the same!")
else:
    print("\tThere is somthing wrong with the conditions!")

Is there any False in dCond: True
Is there any True in dCond: True
Is there any False in sCond: True
Is there any True in sCond: True
Testing equality:
	The conditions are the same!


Build cond arrays
----------

In [103]:
start = time.time()
Xdc = Xd[dCond]
times['dEvalCondX'] = time.time() - start
sizes['dEvalCondX'] = getsizeof(Xdc)
start = time.time()
Ydc = Yd[dCond]
times['dEvalCondY'] = time.time() - start
sizes['dEvalCondY'] = getsizeof(Ydc)
start = time.time()
Zdc = Zd[dCond]
times['dEvalCondZ'] = time.time() - start
sizes['dEvalCondZ'] = getsizeof(Zdc)

times['dEvalCond'] = times['dEvalCondX'] +times['dEvalCondY'] +times['dEvalCondZ'] 
sizes['dEvalCond'] = sizes['dEvalCondZ'] + sizes['dEvalCondY'] + sizes['dEvalCondZ']

print("Xc shape:",Xdc.shape)
print("Yc shape:",Ydc.shape)
print("Zc shape:",Zdc.shape)
print (Zdc)

Xc shape: (150,)
Yc shape: (150,)
Zc shape: (150,)
[ 0.09090909  0.09090909  0.09090909  0.27272727  0.27272727  0.27272727
  0.45454545  0.45454545  0.45454545  0.63636364  0.63636364  0.63636364
  0.81818182  0.81818182  0.81818182  0.18181818  0.18181818  0.18181818
  0.54545455  0.54545455  0.54545455  0.90909091  0.90909091  0.90909091
  1.27272727  1.27272727  1.27272727  1.63636364  1.63636364  1.63636364
  0.27272727  0.27272727  0.27272727  0.81818182  0.81818182  0.81818182
  1.36363636  1.36363636  1.36363636  1.90909091  1.90909091  1.90909091
  2.45454545  2.45454545  2.45454545  0.36363636  0.36363636  0.36363636
  1.09090909  1.09090909  1.09090909  1.81818182  1.81818182  1.81818182
  2.54545455  2.54545455  2.54545455  3.27272727  3.27272727  3.27272727
  0.45454545  0.45454545  0.45454545  1.36363636  1.36363636  1.36363636
  2.27272727  2.27272727  2.27272727  3.18181818  3.18181818  3.18181818
  4.09090909  4.09090909  4.09090909  0.54545455  0.54545455  0.54545455


In [124]:
start = time.time()
condInd = np.where(sCond)
times['sEvalCondOverhead'] = time.time() - start
sizes['sEvalCondOverhead'] = getsizeof(condInd)

print (condInd)

start = time.time()
Xsc = Xs[sCond]
times['sEvalCondX'] = time.time() - start
sizes['sEvalCondX'] = getsizeof(Xsc)

start = time.time()
Ysc = Ys[sCond]
times['sEvalCondY'] = time.time() - start
sizes['sEvalCondY'] = getsizeof(Ysc)

start = time.time()
#Zsc = Zs[condInd[:2]][:,0]
print (sCond[np.where(sCondZ)].shape)#.reshape(sCondZ.shape))
Zsc = Zs[sCond[np.where(sCondZ)]]
times['sEvalCondZ'] = time.time() - start
sizes['sEvalCondZ'] = getsizeof(Zsc)

times['sEvalCond'] = times['sEvalCondX'] +times['sEvalCondY'] +times['sEvalCondZ'] +times['sEvalCondOverhead']
sizes['sEvalCond'] = sizes['sEvalCondZ'] + sizes['sEvalCondY'] + sizes['sEvalCondZ'] + sizes['sEvalCondOverhead']


print("Xc shape:",Xsc.shape)
print("Yc shape:",Ysc.shape)
print("Zc shape:",Zsc.shape)
print (Zsc)

(array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]), array([ 6,  6,  6,  7,  7,  7,  8,  8,  8,  9,  9,  9, 10, 10, 10,  6,  6,
        6,  7,  7,  7,  8,  8,  8,  9,  9,  9, 10, 10, 10,  6,  6,  6,  7,
        7,  7,  8,  8,  8,  9,  9,  9, 10, 10, 10,  6,  6,  6,  7,  7,  7,
        8,  8,  8,  9,  9,  9, 10, 10, 10,  6,  6,  6,  7,  7,  7,  8,  8,
        8,  9,  9,  9, 10, 10, 10,

IndexError: boolean index did not match indexed array along dimension 0; dimension is 11 but corresponding boolean dimension is 60

Compare cond arrays
-----------

In [106]:
if np.array_equiv(Xdc,Xsc):
    print("\tThe condArrayX are the same!")
else:
    print("\tThere is somthing wrong with the condArrayX!")
    #for i,eld in enumerate(Xdc):
    #    print("el:",i)
    #    print("\tXdc:",eld)
    #    print("\tXsc:",Xsc[i])

if np.array_equiv(Ydc,Ysc):
    print("\tThe condArrayY are the same!")
else:
    print("\tThere is somthing wrong with the condArrayY!")
    

if np.array_equiv(Zdc,Zsc):
    print("\tThe condArrayZ are the same!")
else:
    print("\tThere is somthing wrong with the condArrayZ!")

	The condArrayX are the same!
	The condArrayY are the same!
	There is somthing wrong with the condArrayZ!


In [25]:
print('TIMES\n',"-"*20)
timesKeys = times.keys()
for i,el in enumerate(timesKeys):
    timesKeys[i]=el[1:]
timesKeys = list(dict.fromkeys(timesKeys))

fullTd = 0.0
fullTs= 0.0

for el in timesKeys:
    print(el)
    print("\ts",times['s'+el])
    fullTs+=times['s'+el]
    try:
        print("\td",times['d'+el])
        fullTd += times['d'+el]
    except:
        pass

print('full time default: %1.4e'%fullTd)
print('full time sparse:  %1.4e'%(fullTs-times['sEvalCondOverhead']))


print('SIZES\n',"-"*20)
sizesKeys = sizes.keys()
for i,el in enumerate(sizesKeys):
    sizesKeys[i]=el[1:]
sizesKeys = list(dict.fromkeys(sizesKeys))

for el in sizesKeys:
    print(el)
    print("\ts",sizes['s'+el])
    try:
        print("\td",sizes['d'+el])
    except:
        pass

TIMES
 --------------------
EvalCondZ
	s 0.00149393081665
	d 0.00300908088684
EvalCondY
	s 0.00163602828979
	d 0.0108368396759
EvalCondX
	s 0.00218105316162
	d 0.0027379989624
EvalCondOverhead
	s 0.0111010074615
Mesh
	s 0.000253200531006
	d 0.0370881557465
Cond
	s 0.00744009017944
	d 0.0164179801941
EvalCond
	s 0.0164120197296
	d 0.0165839195251
Trafo
	s 0.0306460857391
	d 0.359937906265
full time default: 4.4661e-01
full time sparse:  6.0062e-02
SIZES
 --------------------
MeshC
	s 944
	d 8488976
EvalCondZ
	s 1000096
	d 1000096
EvalCondY
	s 1000096
	d 1000096
EvalCondX
	s 1000096
	d 1000096
EvalCondOverhead
	s 80
MeshP
	s 952
	d 8488976
MeshR
	s 936
	d 8488976
Cond
	s 1061234
	d 1061234
EvalCond
	s 3000368
	d 3000288
MeshX
	s 8488976
	d 8488976
MeshY
	s 8488976
	d 8488976
MeshZ
	s 82544
	d 8488976
